*AGEC 652 - Spring 2023*

*Unit 3.2 - Nonlinear equations*

# Solving a spatial general equilibrium model in Julia

In this exercise, we will solve a 2-country example of the general model from the slides. This will be an opportunity to use many of the ideas we saw in this unit.

## Setting
### Demand and supply
There are two countries $i \in {1,2}$ and a single commodity. Both countries have an isoelastic demand given by
$$D_i(p_i) = \alpha p_i^{\eta_i}$$
and isolastic supply given by
$$S_i(p_i) = \beta p_i^{\rho_i}$$

In this exercise, our model parameters are
$$\alpha_1 = 5, \alpha_2 = 3 \\
\beta_1 = 2, \beta_2 = 6 \\
\eta_1 = 1.2, \eta_2 = 1.5 \\
\rho_1 = 1.6, \rho_2 = 1.2$$

In [1]:
# Model paremeters
α = [5.0; 3.0];
β = [2.0; 6.0];
η = [1.2; 1.5];
ρ = [1.6; 1.2];

### Trade frictions
Trade between countries incur in symmetric transportation costs $c_{12} = c_{21} = 0.2$.

Moreover, transportation capacity is limited and the maximum amount of trade flow in any direction is $b = 1.5$. Of course, trade flows are non-negative.

### Problem
We want to find the market equilibrium given by $q_1, q_2, p_1, p_2$ such that both markets clear.

**This exercise:** We will start with the simplest version of the model and add elements until solve the whole problem.

## Step 1: No trade

**Q: How do we solve the model when there is no trade (i.e., autarky)?**

(This is the equivalent to assuming $c_{12} = c_{21} = \infty$)

A: We just need to solve two separate problems of $D_i(p_i) = S_i(p_i)$

Following the slides, it'll be easier to define a single excess demand function given by 
$$E_i(p_i) \equiv D_i(p_i) - S_i(p_i) = \alpha p_i^{\eta_i} - \beta p_i^{\rho_i}$$

Let's define this as a vector function and test run it.

In [2]:
E(p) = α .* p.^(-η) - β .* p.^(ρ);
E([0.5; 0.5])  

2-element Vector{Float64}:
 10.827229594583901
  5.873629684350199

So, to solve the no-trade version of the model, we need to find vector $(p_1, p_2)$ that solves $E_1(p_1) = 0$ and $E_2(p_2) = 0$.

#### YOUR TURN: solve the no-trade model using NLsolve.nlsolve and calculate $p$ and $E$ for each country.

(By the way: can you solve it analytically?)

In [3]:
using NLsolve;
no_trade_results = NLsolve.nlsolve(E, [0.5; 0.5], method=:newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.5, 0.5]
 * Zero: [1.3871436293699804, 0.7735838759133007]
 * Inf-norm of residuals: 0.000000
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 6
 * Jacobian Calls (df/dx): 6

We can get the price

In [4]:
no_trade_p = no_trade_results.zero

2-element Vector{Float64}:
 1.3871436293699804
 0.7735838759133007

So country 1 has a higher price and there are potential gains from trade.

We can check excess demand to see if this is indeed a valid solution.

In [5]:
no_trade_E = E(no_trade_p)

2-element Vector{Float64}:
 3.6921576906934206e-11
 0.0

## Step 2: integrated economies with no trade frictions

Now we go a step further and assume both countries can trade freely and without any transportation costs (i.e., we assume $c_{12} = c_{21} = 0$) and no trade flow constraints.

**Q: How do we solve the model with fully integrated economies without trade frictions?**

Without trade frictions, an equilibrium with fully integrated economies will be just like the case when they are the same country. Hence, the law of one price will hold: there is a single price that clears the joint market.
$$ E_1(p) + E_2(p) = 0$$

#### YOUR TURN: solve the no-trade-friction model using NLsolve.nlsolve and calculate $p$ and $E$ for each country.

All we have to do is define $f(p) = E_1(p) + E_2(p)$ and pass it to `nlsolve`.

In [6]:
f_integr(p) = sum(E(p));
integ_results = NLsolve.nlsolve(f_integr, [0.9], method=:newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.9]
 * Zero: [0.9999999998477678]
 * Inf-norm of residuals: 0.000000
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 4

We can store the equilibrium price...

In [7]:
integ_p = integ_results.zero

1-element Vector{Float64}:
 0.9999999998477678

and check what the excess demands are

In [8]:
integ_E = E(integ_p)

2-element Vector{Float64}:
  3.000000001400537
 -2.999999998218883

So, if both economies are fully integrated, the resulting price is 1 (which is between both autarky prices) and **country 1 imports 3 units from country 2**.

## Step 3: unconstrained trade flows

Now we bring back transportation costs but leave trade flow constraints out. Let's define the transportation cost vector.

In [9]:
#   [c_12; c_21]
c = [0.2; 0.2]

2-element Vector{Float64}:
 0.2
 0.2

Because of transportation costs, we know that prices within each country will not be equalized.

**Q: How do we solve the model with transportation costs but no trade flow constraints?**

We are still looking for a price vector that satisfies market conditions. The first condition is easy because it's the same as before: markets have to clear
$$f_1(p_1, p_2) = E_1(p_1) + E_2(p_2) = 0$$

For the second condition, we need to think about arbitrage opportunities. We know that if the price in one country is greater than the other, there is an incentive to import. 
- Imports will raise the price in the other country
- This process will continue until there is no arbitrage: the domestic price is the same as the foreign price plus the transportation cost 

However, the problem here is that this condition depends on who is the net importer. We already expect it to be country 1, but we can define a flexible $f_2$ to handle it

$$
f_{2}\left(p_{1},p_{2}\right)=\begin{cases}
p_{1}-p_{2}-c_{21} & \text{if}p_{1}\geq p_{2}\\
p_{2}-p_{1}-c_{12} & \text{if}p_{1}<p_{2}
\end{cases}$$

#### YOUR TURN: solve the unconstrained flow model using NLsolve.nlsolve and calculate $p$ and $E$ for each country.

We can define an `f` with an `if` clause

In [10]:
function f_unc_trade!(F, p)
    # E_1 + E_2 = 0
    F[1] = sum(E(p))
    if (p[1] >= p[2])
        # p_1 = p_2 + c_21
        F[2] = p[1] - p[2] - c[2]
    else
        # p_2 = p_1 + c_12
        F[2] = p[2] - p[1] - c[1]
    end
end;

Note how I coded the function to be memory efficient by modifying a pre-allocated vector `F`.

This function has a kink, so it's not guaranteed to converge! However, it's not guaranteed to diverge either, so we can try our luck. 
- We could have analyzed the model without trade costs (like step 2) and made an educated guess that $p_1 > p_2$ because we know that country 1 is the importer
- Another approach would be to guess one of the cases, try to get a solution and check whether it's actually a solution. If we guess wrong, we might not have a solution 

Now we solve it

In [11]:
unc_trade_results = NLsolve.nlsolve(f_unc_trade!, [1.1; 0.9], method=:newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [1.1, 0.9]
 * Zero: [1.116994073995761, 0.9169940739957608]
 * Inf-norm of residuals: 0.000000
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 4

And get the price vector

In [12]:
unc_trade_p = unc_trade_results.zero

2-element Vector{Float64}:
 1.116994073995761
 0.9169940739957608

To which we can check that the price differential is indeed equal to transportation cost

In [13]:
unc_trade_p[1] - unc_trade_p[2]

0.20000000000000007

We can also get E

In [14]:
unc_trade_E = E(unc_trade_p)

2-element Vector{Float64}:
  1.9910106139888089
 -1.991010613988808

So, with transportation costs, **country 1 imports about 1.99 units from country 2**.

We can also verify this is indeed an equilibrium

In [15]:
F = zeros(2);
f_unc_trade!(F, unc_trade_results.zero);
F

2-element Vector{Float64}:
 8.881784197001252e-16
 5.551115123125783e-17

## Step 4: full model

Now we are ready for the grand finale: we add trade flow constraints $b_{ij}$. We know that this will involve a complementarity problem. 

Let's use $x_{ij}$ to denote the trade flow from country $i$ to country $j$. 
- This is, country $j$ is the importer and $i$ the exporter.

Also, trade flows are non-negative, so $0 \leq x_{ij} \leq b_{ij}$

We know that if trade flows are positive, then domestic prices are **at least** as high as the price of buying a foreign good (foreign price + transportation cost). This gives

$$x_{ij} > 0 \Rightarrow p_j - p_i - c_{ij} \geq 0 $$

We also know that if trade flows are below trade limits, then there are no arbitrage opportunities left and domestic prices are **at most** as high as the price of importing. This gives

$$x_{ij} < b_{ij} \Rightarrow p_j - p_i - c_{ij} \leq 0 $$

But hold on. We've been solving everything in terms of prices but constraints are on trade flows so far. 

**How do we go from solving for prices to solve for trade flows?**

Following the general example in the slides, we need to relate trade flows and prices.

We know that market clearing in each region $i$ requires that net imports = excess demand, so

$$\sum_k [x_{ki} - x_{ik}] = E_i(p_i)$$

So we can solve for prices

$$p_i = E_i^{-1}\left( \sum_k [x_{ki} - x_{ik}]\right)$$

That is, if $E_i^{-1}$ exists... does it?

- Yes! At least for the domain we care about: $E_i(p_i)$ is monotonic for $p_i>0$

So now we can go back to our complementarity conditions and plug in prices

$$
f_{21}(x) = E_1^{-1}\left( x_{21} - x_{12} \right) - E_2^{-1}\left(x_{21} - x_{12}\right) - c_{21} \\
f_{12}(x) = E_2^{-1}\left( x_{12} - x_{21} \right) - E_1^{-1}\left(x_{12} - x_{21}\right) - c_{12}
$$

We have $f$, $a$ and $b$, so we're good to go.
- Note that $f_{11}$ and $f_{22}$ are trivially equal zero, so we don't need to solve for $x_{11}$ or $x_{22}$

#### YOUR TURN: solve the constrained flow model using NLsolve.mcpsolve and calculate $p$ and $E$ for each country.

But hold on! How the hell do we calculate $E_i^{-1}$?

Great question! Let's see... Can you invert $E_i$ analytically?


Nope... we need to do it numerically.

With $p_i = E_i^{-1}$, we have some excess demand/net trade flow $x$ and we want to find $p$ such that $E(p) = x$. How do we find $p$?

- How about we define some $g(p) = E(p) - x$? Then it's a nonlinear rootfinding problem!

We will define it to return a scalar. That's just to make our life easier when coding $f$.

In [16]:
function E_inv(x, index)
    g(p) = E(p)[index] .- x
    NLsolve.nlsolve(g, [1.0], method=:newton).zero[1]
end;

Let's test run our function. If we plug in zero excess demand, we know what we should get, right?

In [17]:
E_inv(0.0, 1)

1.3871436293753914

In [18]:
E_inv(0.0, 2)

0.773583875913069

OK, now we are ready to continue

#### YOUR TURN: solve the constrained flow model using `NLsolve.mcpsolve` and calculate $p$ and $E$ for each country.

(When using `mcpsolve`, remember that standard MCP problem has the opposite signs from Miranda and Fackler's (and from our lectures). An easy solution is to just define our $f$ with a flipped sign.)

First, we define $f$

In [29]:
function f_const_trade!(F, x)
    # Solve for prices using trade flows
    # p1 = E_inv_1(x_21 - x_12)
    p1 = E_inv(x[2] - x[1], 1)
    # p2 = E_inv_2(x_12 - x_21)
    p2 = E_inv(x[1] - x[2], 2)
    #f_12: p2 - p1 - c_12
    F[1] = -(p2 - p1 - c[1]) # Flip sign for mcpsolve
    #f_21: p1 - p2 - c_21
    F[2] = -(p1 - p2 - c[2]) # Flip sign for mcpsolve
end;

Then, define lower and upper bounds and run the solution

In [30]:
a = [0.0; 0.0]
b = [1.5; 1.5]

const_trade_results = NLsolve.mcpsolve(f_const_trade!, a, b, [3.0; 0.0], method=:newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [3.0, 0.0]
 * Zero: [-4.430784348463855e-17, 1.5000000009771264]
 * Inf-norm of residuals: 0.000000
 * Iterations: 7
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 9
 * Jacobian Calls (df/dx): 7

In [31]:
const_trade_x = const_trade_results.zero

2-element Vector{Float64}:
 -4.430784348463855e-17
  1.5000000009771264

So both constraints bind: **country 1 imports at maximum capacity (`x21 = 1.5`) from country 2**

Let's check prices

In [33]:
const_trade_p = [E_inv(const_trade_x[2]-const_trade_x[1], 1); E_inv(const_trade_x[1]-const_trade_x[2], 2)]

2-element Vector{Float64}:
 1.1790808075246417
 0.879045725555841

In [34]:
const_trade_p[1] - const_trade_p[2]

0.3000350819688007

So the price differentials exceed transportation costs because trade flow constraints prevent price equalization

We can also check excess demands

In [35]:
const_trade_E = E(const_trade_p)

2-element Vector{Float64}:
  1.500000000977126
 -1.4999999925872487

So, indeed trade flows offset excess demands.

We can also check what is happening to $f$

In [36]:
F = zeros(2);
f_const_trade!(F, const_trade_x);
-F # We need to re-flip the sign coming out of mcpsolve

2-element Vector{Float64}:
 -0.5000350819688006
  0.10003508196880068

Since both constraints are binding, $f$ is non-zero in both cases. 
- $f_{21} \approx 0.1 > 0$ indicates that arbitrage opportunities would exist by importing more if we were not constrained with flow limits
- $f_{12} \approx -0.5 < 0$ is the opposite: country 2 could benefit from exporting more if it were not constrained